# 新闻聚类分群模型

新闻种类繁复多样，可以分为财经、体育、科技、娱乐等题材。在本案例中，根据10个关键词从百度新闻爬取962条新闻，且每个关键词对应的新闻条数相近，通过Python编程对每条新闻划分类别，匹配到正确的版面，这实际上就是在对新闻进行聚类分群。

## 一、文本数据的读取与处理
### 1、读取数据

In [1]:
import pandas as pd
df = pd.read_excel('datasets/新闻.xlsx')
df.head()

,关键词,标题,网址,来源,时间
0,华能信托,信托公司2019年上半年经营业绩概览,http://www.financialnews.com.cn/jrsb_m/xt/zx/2...,中国金融新闻网,2019年07月23日 00:00
1,华能信托,首单信托型企业ABS获批,http://www.jjckb.cn/2018-10/23/c_137552198.htm,经济参考网,2018年10月23日 12:21
2,华能信托,华能贵诚信托孙磊:金融科技助力打造开放信托生态,https://baijiahao.baidu.com/s?id=1639276579449...,同花顺财经,2019年07月17日 10:49
3,华能信托,华能贵诚信托孙磊:金融科技已经成为信托行业重要的基础设施,https://finance.qq.com/a/20190716/007898.htm,腾讯财经,2019年07月16日 18:53
4,华能信托,格力电器股权转让意向方闭门开会 华能信托赫然在列,https://finance.sina.com.cn/trust/roll/2019-05...,新浪,2019年05月22日 22:53


### 2、中文分词

> 通过 jieba 库进行中文分词练习

In [3]:
import jieba
word = jieba.cut('我喜欢编写程序')
for i in word:
    print(i)

我
喜欢
编写程序


In [4]:
# 第1条新闻的标题进行分词
import jieba
word = jieba.cut(df.iloc[0]['标题']) #用cut()函数对第1条新闻的标题进行分词，并将分词结果赋给变量word，其中df.iloc[0]['标题']表示选取第1条新闻的标题（iloc[0]表示选取第1行数据，即第1条新闻的数据，['标题']表示选取列名为“标题”的列）
result = ' '.join(word)
print(result)

信托公司 2019 年 上半年 经营 业绩 概览


#### 对所有的新闻标题进行分词

In [5]:
words = [] # 创建一个空列表words来存储每一条新闻标题的分词结果
for i, row in df.iterrows(): # 通过for循环遍历整张表格，其中df.iterrows()是pandas库遍历表格每一行的方法，i对应每一行的行号，row对应每一行的内容
    word = jieba.cut(row['标题'])
    result = ' '.join(word)
    words.append(result) # 用append()函数将每一条新闻标题的分词结果添加到words列表中

In [8]:
# 熟悉上述代码后，也可以将其合并成下面的3行代码
words = []
for i, row in df.iterrows():
    words.append(' '.join(jieba.cut(row['标题'])))
words[0:5]

['信托公司 2019 年 上半年 经营 业绩 概览',
 '首单 信托 型 企业 ABS 获批',
 '华能 贵 诚信 托孙磊 : 金融 科技 助力 打造 开放 信托 生态',
 '华能 贵 诚信 托孙磊 : 金融 科技 已经 成为 信托 行业 重要 的 基础设施',
 '格力电器 股权 转让 意向 方 闭门 开会   华能 信托 赫然 在 列']

### 3、文本向量化：构造特征变量
> 练习：文本向量化基础：建立词频矩阵

In [22]:
from sklearn.feature_extraction.text import CountVectorizer # 从Scikit-Learn库中引入CountVectorizer模块
test = ['金融 科技 业绩', '华能 信托 业绩'] # 用于演示的新闻标题，共有2条
vect = CountVectorizer() # 将CountVectorizer()函数赋给变量vect
x= vect.fit_transform(test) # 用fit_transform()函数进行文本向量化转换，并将转换结果赋给变量x
x = x.toarray() # 用toarray()函数将x转换为数组
x

array([[1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0]])

此时2条新闻标题已经变成了由数字0和1组成的2个一维数组，每个数组中各有5个元素。CountVectorizer()函数会先根据空格来识别每一句话中的词语，例如，它能从第1条新闻标题中识别出“金融”“科技”“业绩”这3个词，从第2条新闻标题中识别出“华能”“信托”“业绩”这3个词，这2条标题一共能识别出“金融”“科技”“华能”“信托”“业绩”这5个不同的词。这5个词便构成了这2条新闻标题的词袋，CountVectorizer()函数会自动对词袋中的词进行编号，通过vocabulary_属性便能获取词袋的内容及相应编号，如下：

In [23]:
words_bag = vect.vocabulary_
print(words_bag)

{'金融': 4, '科技': 3, '业绩': 0, '华能': 2, '信托': 1}


如果只想看到词，不想看到编号，可以使用如下代码

In [24]:
words_bag2 = vect.get_feature_names()
print(words_bag2)

['业绩', '信托', '华能', '科技', '金融']


构建词频矩阵，表中的数值即为相关标题中对应特征词的出现频次
CountVectorizer()函数会自动过滤掉一个字的词，这样会过滤掉“的”“之”等没有重要含义的词，不过同时也会过滤掉“爱”“坑”等可能有重要含义的词，因此，这个特点既是一个优势，也是一个劣势。

In [26]:
import pandas as pd
df = pd.DataFrame(x, columns=words_bag2)
df.head()

,业绩,信托,华能,科技,金融
0,1,0,0,1,1
1,1,1,1,0,0


#### 文本向量化实战：建立词频矩阵

In [28]:
# 将所有的新闻标题进行文本向量化并利用pandas库进行展示
from sklearn.feature_extraction.text import CountVectorizer # 从Scikit-Learn库引入CountVectorizer模块
import pandas as pd
vect = CountVectorizer() # 将CountVectorizer()函数赋给变量vect
x = vect.fit_transform(words) # 用fit_transform()函数进行文本向量化处理并赋给变量x
x = x.toarray() # 用toarray()函数将X转换为数组形式并重新赋给变量x
word_bag3 = vect.get_feature_names() # 从词袋中获取不带编号的词
df = pd.DataFrame(x, columns=word_bag3)
df.head()

/Users/dayao/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00700,03,04,08s,09,10,100,11,12,150,...,黄萍,黄金,黑客,黑灰产,黑金,黑马,鼓手,鼻祖,齐聚,龙风
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


总结来说，当有n条新闻标题时，先用jieba库对它们进行分词，然后用CountVectorizer()函数提取所有分词中k个不同的词，用这些词构成一个词袋，每个词对应一个编号，即相应的特征，根据原标题中相关词出现的次数来赋值相关特征为i（即相关词出现的次数），这样就完成了文本数值化的工作，接下来就可以进行模型的搭建与使用了。

## 二、模型的搭建与使用

### 1、通过KMeans算法进行聚类分群
本案例的原始数据是根据10个关键词爬取的新闻。下面先利用KMeans算法搭建模型进行聚类分群，看看它能否将来自10个不同题材的新闻准确地分门别类

In [29]:
from sklearn.cluster import KMeans  # 引入Scikit-Learn库中的KMeans模型
kms = KMeans(n_clusters=10, random_state=123) # 设置KMeans模型的n_clusters参数为10，即将样本聚成10类，并设置random_state参数为123，使得每次运行代码获得的聚类结果保持一致
k_data = kms.fit_predict(df) # 用fit_predict()函数将模型拟合训练和聚类结果传递合二为一(等同于先用fit()函数训练模型，再通过labels_属性获取聚类结果的写法)
k_data # 其中每个数字代表一个分类，可以看到，KMeans算法将新闻标题分成了10类

array([0, 0, 3, 3, 0, 0, 0, 0, 0, 8, 0, 8, 8, 0, 0, 0, 0, 8, 0, 0, 8, 0,
       7, 0, 0, 0, 0, 0, 0, 0, 8, 0, 8, 0, 8, 0, 0, 0, 7, 8, 0, 8, 0, 8,
       0, 0, 0, 7, 8, 0, 8, 0, 8, 0, 0, 0, 7, 8, 0, 8, 0, 8, 0, 0, 0, 7,
       8, 0, 8, 0, 8, 0, 0, 0, 7, 8, 0, 8, 0, 8, 0, 0, 0, 7, 8, 0, 8, 0,
       8, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3,
       3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3,
       3, 9, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 0, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3,
       3, 3, 0, 3, 3, 9, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 0, 3, 3, 3, 3, 2, 0, 0, 2, 2, 0, 2, 2,

In [33]:
# 打印分类为1的数据查看该分类中的新闻标题
import numpy as np
words_ary = np.array(words)
print(words_ary[k_data == 6])

['阿里巴巴 的 食堂 长 啥 样 ? 上海 网红 美食 杭州 首站 开在 这'
 '阿里巴巴 朋新宇 : 为何 中台 能 帮 企业 突破 增长 瓶颈 ?'
 '阿里巴巴 搭 起 数字 鹊桥 : 4000 万 人种 下 情侣 树   600 万人 绑定 淘宝 亲情 号'
 '阿里巴巴 大 数据 折射 中国式 “ 我爱你 ” : 54% 码商 夫妻店 由 老婆 管帐'
 '阿里巴巴 合伙人 之一   跟 了 马云 19 年 后   分 了 40 亿给 她'
 '三年 薪酬 1.2 亿   成 中国 上市公司 CEO 第一   超过 阿里巴巴 张勇'
 '投资者 提问 : 阿里巴巴 平头 哥 宣布 “ 普惠 芯片 ” 计划 。 即 未来 将 全面 ...'
 '探秘 阿里巴巴 未来 酒店 健康 空气 之源' '思必驰 与 华为 、 阿里巴巴 等 人工智能 企业 获评 人工智能 重大 创新 工程 ...'
 '玄武区 白酒 贴牌 定制 选择 和 阿里巴巴 合作 的' '兴化 阿里巴巴 托管 技术'
 '美国 股指 期货 反弹   阿里巴巴 集团 涨 2.3%' '铜川 阿里巴巴 账号 优化'
 '美股 全线 高开 , 标普 500 指数 涨 0.7% , 中概 股 反弹 , 阿里巴巴 涨 逾 3%'
 '中国 盈利 最高 的 公司 , 不是 阿里巴巴 , 也 不是 腾讯 , 第一名 实至名归'
 '阿里巴巴 成为 申通 第一 大 股东 , “ 快递 女王 ” 离开 , 网友 : 服 了 !'
 '申通 大 股东 将 全部 股权 出质 给 阿里巴巴' '携手 阿里巴巴   福建 东山 办开 渔节'
 '中概 股盘 前 走高   阿里巴巴 集团 涨超 2%'
 '投资者 提问 : 董秘 您好 , 近期 有 媒体 传闻 , 阿里巴巴 要 战略 投资 贵 公司 , ...'
 '东榜 投资 引入 阿里巴巴 集团 体系 , 加速 东山岛 互联网 + 新 渔港 经济体 进程'
 '阿里巴巴 的 顶层 “ 秘密组织 ” 成员 过百 ! 马云 退出 后 , 将 进一步 壮大'
 '阿里巴巴 的 “ 拖油瓶 ” 项目 ? 一年 亏了 200 多亿 , 马云 表示 无所谓'
 '走进 阿里巴巴 开年 中 总结 大会 , 万事利 集团 未来 要 以 数据 与 智能 驱动 ...'
 '第二届 

## 三、模型优化